In [2]:
import netCDF4 as nc
from metpy.plots import USCOUNTIES
import metpy
import cartopy.crs as ccrs 
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import wrf
from wrf import vinterp, to_np, getvar, CoordPair, vertcross, get_cartopy, latlon_coords
import pandas as pd
import matplotlib.ticker as ticker
import os
from metpy.units import units 
import metpy.calc as mpcalc
import xarray as xr

C:\Users\Tony\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
ncfile = nc.Dataset('c:\\Users\\Tony\\Desktop\\wrfoutput\\ctrl\\wrfout_d02_2019-07-20_03_00_00')

In [4]:
t = getvar(ncfile, "temp", units='k') * units.kelvin
td = getvar(ncfile, "td", units='k') * units.kelvin
p = getvar(ncfile, "pressure") * units.hPa
z = getvar(ncfile, "z", units='m') * units.meter

p_sliced = p[0:25, :, :]
t_sliced = t[0:25, :, :]
td_sliced = td[0:25, :, :]

lfc_array = np.empty((p_sliced.shape[0], p_sliced.shape[1], p_sliced.shape[2]))

In [5]:
for i in range (p_sliced.shape[0]):
    for j in range (p_sliced.shape[1]):
        for k in range (p_sliced.shape[2]):
            try:
                lfc_pressure, lfc_temperature = metpy.calc.lfc(p_sliced[i:,j,k].values*units.hPa, t_sliced[i:,j,k].values*units.kelvin, td_sliced[i:,j,k].values*units.kelvin)
                lfc_array[i,j,k] = lfc_pressure.magnitude 
                #print(lfc_pressure.magnitude)
            except:
                lfc_array[i,j,k] = np.nan

lfc_da1 = xr.DataArray(lfc_array, dims=['top_bottom', 'north_south', 'east_west'])

C:\Users\Tony\AppData\Local\Temp\ipykernel_25204\2326569300.py:5: UserWarning: Interpolation point out of data bounds encountered
  lfc_pressure, lfc_temperature = metpy.calc.lfc(p_sliced[i:,j,k].values*units.hPa, t_sliced[i:,j,k].values*units.kelvin, td_sliced[i:,j,k].values*units.kelvin)
C:\Users\Tony\anaconda3\lib\site-packages\metpy\interpolate\one_dimension.py:160: RuntimeWarning: divide by zero encountered in divide
  var_interp = var[below] + (var[above] - var[below]) * ((x_array - xp[below])
C:\Users\Tony\anaconda3\lib\site-packages\pint\facets\plain\quantity.py:976: RuntimeWarning: invalid value encountered in multiply
  magnitude = magnitude_op(self._magnitude, other_magnitude)


In [6]:
path = 'C:\\Users\\Tony\\Desktop\\wrfoutput'
file_name = 'lfc_test_0300.npy'

# Save the array to a .npy file
np.save(file_name, lfc_da1)